In [1]:
import re
import os
import pandas as pd
from heapq import nlargest
from collections import Counter
from string import punctuation
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup



In [288]:
master_array = [] #append results to array to create data frame


for filename in os.listdir('seasons/'):
    
    if filename.endswith(".html"):
        
        split_name = filename.split('.') #obtain the season and episodes
        season = split_name[0]   
        episode = split_name[1]

        master_array.append([season, episode])
    else:
        pass
    
# Skapar nycklar till lexikon som sedan ska göras om till en dataframe
numblines = dict()
numblines["Episode"] = []
numblines["Number of lines"] = []
numblines["Number of words"] = []
numblines["Different words used"] = []
numblines["Words per line"] = []
numblines["Different words ratio"] = []
numblines["Exclamation marks"] = []
numblines["Question marks"] = []
numblines["Word"] = []
numblines["Most common Monica"] = []
numblines["Most common Rachel"] = []
numblines["Most common Phoebe"] = []
numblines["Most common Chandler"] = []
numblines["Most common Ross"] = []
numblines["Most common Joey"] = []
numblines["Character"] = []
    
for num, ep in enumerate(master_array):

    filename=master_array[num][0]+'.'+master_array[num][1]
    f = open('seasons/'+filename, 'r')
    data = f.read()

    soup = BeautifulSoup(data)
    soup = soup.get_text().translate(str.maketrans({'\n': ' ', '\xa0': ''}))
    pattern = re.compile(r'\s(?=\w+(?=:))') # store the regex
    result = re.split(pattern, soup) # split the script where our pattern matched (pink dot)

    # Fördela datan i karaktär och replik
    episode_array = []
    for item in result:
        split_line = item.split(': ')
        try:
            character = split_line[0]
            speech = split_line[1]
            episode_array.append([character, speech])
        except:
            pass
    
    df = pd.DataFrame(episode_array, columns = ['Person','Said'])
    df['Person'].replace({'CHAN':'Chandler','CHANDLER':'Chandler', 'Chandlers':'Chandler',
                        'JOEY':'Joey',
                        'MNCA':'Monica','MONICA':'Monica',
                        'PHOE':'Phoebe','PHOEBE':'Phoebe', 'Pheebs':'Phoebe',
                        'Rache':'Rachel','RACHEL':'Rachel', 'RACH':'Rachel',
                        'ROSS':'Ross'},inplace=True)

    
    # Tar bort (), [], och "[Scene" från datan
    curved = re.compile("(\(.*?\))")
    brackets = re.compile("(\[.*?\])")
    scene = re.compile("\[Scene")
    for j,i in enumerate(df['Said']):
        df.iloc[j][1] =re.sub(curved, '', i)

    for j,i in enumerate(df['Said']):
        df.iloc[j][1] =re.sub(brackets, '', i)
        
    for j,i in enumerate(df['Said']):
        df.iloc[j][1] =re.sub(scene, '', i)

    # Lista med unika ord samt antalet de sägs
    worddicts = {}
    worddicts['Monica'] = {}
    worddicts["Rachel"]={}
    worddicts["Phoebe"]={}
    worddicts["Chandler"]={} 
    worddicts["Ross"]={} 
    worddicts["Joey"]={}
    for i, j in enumerate(df["Person"]):
        if j in worddicts:
            wordlist = Counter((x.rstrip(punctuation).lower() for x in df.iloc[i][1].split()))
            for word in wordlist:
                try:
                    worddicts[j][word] = worddicts[j][word]+1
                except KeyError:
                    worddicts[j][word] = 1
    
    # Skapar dictionary för antal rader, frågetecken och utropstecken
    namedict=dict()
    namedict["Rachel"]=[0]*3
    namedict["Phoebe"]=[0]*3
    namedict["Chandler"]=[0]*3
    namedict["Ross"]=[0]*3
    namedict["Joey"]=[0]*3
    namedict["Monica"]=[0]*3
    
    k = 0
    for i in df["Person"]:
        if i in namedict:
            try:
                namedict[i][0] = namedict[i][0]+1
                namedict[i][1] = namedict[i][1]+Counter(df.iloc[k][1])['!']
                namedict[i][2] = namedict[i][2]+Counter(df.iloc[k][1])['?']
            except:
                pass
        k = k+1
    
    # Ny kod för unika ord
    ucommonwords = {}
    ucommonwords['Monica']={}
    ucommonwords["Rachel"]={}
    ucommonwords["Phoebe"]={}
    ucommonwords["Chandler"]={} 
    ucommonwords["Ross"]={} 
    ucommonwords["Joey"]={}
    for b in namedict:
        for c in namedict:
            ucommonwords[b][c] = 0
            
    for p in namedict:
        for person in namedict:
            for word in nlargest(30, worddicts[person], key=worddicts[person].get):

                if word in uniqueWord[person]:
                    try:
                        ucommonwords[p][person] = ucommonwords[p][person]+worddicts[person][word]
                    except:
                        ucommonwords[p][person] = worddicts[p][word]
                else:
                    pass
                
        try:
            
            numblines["Number of lines"].append(namedict[p][0])
            numblines["Character"].append(p) 
            numblines["Episode"].append(master_array[num][0])
            #numblines["Most common 10"].append(nlargest(10, worddicts[p], key=worddicts[p].get))
            #numblines["Most common 30"].append(nlargest(30, worddicts[p], key=worddicts[p].get))
            #numblines["Most common 50"].append(nlargest(50, worddicts[p], key=worddicts[p].get))
            numblines["Number of words"].append(sum(worddicts[p].values()))
            numblines["Different words used"].append(len(worddicts[p]))
            numblines["Exclamation marks"].append(namedict[p][1])
            numblines["Question marks"].append(namedict[p][2])
            numblines["Words per line"].append(sum(worddicts[p].values())/namedict[p][0])
            numblines["Different words ratio"].append(len(worddicts[p])/sum(worddicts[p].values()))
            numblines["Most common Monica"].append(ucommonwords[p]["Monica"]/sum(worddicts[p].values()))
            numblines["Most common Rachel"].append(ucommonwords[p]["Rachel"]/sum(worddicts[p].values()))
            numblines["Most common Phoebe"].append(ucommonwords[p]["Phoebe"]/sum(worddicts[p].values()))
            numblines["Most common Chandler"].append(ucommonwords[p]["Chandler"]/sum(worddicts[p].values()))
            numblines["Most common Ross"].append(ucommonwords[p]["Ross"]/sum(worddicts[p].values()))
            numblines["Most common Joey"].append(ucommonwords[p]["Joey"]/sum(worddicts[p].values()))
            try:
                numblines["Word"].append(worddicts[p]["smelly"])
            except:
                numblines["Word"].append(0)
        except:
               pass  
        
epinfo = pd.DataFrame.from_dict(numblines, orient='index')
epinfo.transpose()
    #plt.bar(list(namedict.keys()), namedict.values(), color='g')
    #plt.show()
#print(df.iloc[3][1])    
#print(df.iloc[9][1])

,Episode,Number of lines,Number of words,Different words used,Words per line,Different words ratio,Exclamation marks,Question marks,Word,Most common Monica,Most common Rachel,Most common Phoebe,Most common Chandler,Most common Ross,Most common Joey,Character
0,0101,49,554,277,11.3061,0.5,28,16,0,0,0,0,0,0,0,Rachel
1,0101,19,220,156,11.5789,0.709091,7,4,0,0,0,0,0,0,0,Phoebe
2,0101,39,350,212,8.97436,0.605714,9,8,0,0,0,0,0,0,0,Chandler
3,0101,47,449,265,9.55319,0.5902,14,21,0,0,0,0,0,0,0,Ross
4,0101,41,394,232,9.60976,0.588832,20,16,0,0,0,0,0,0,0,Joey
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1369,1017-1018,85,712,332,8.37647,0.466292,40,33,0,0,0.011236,0,0,0,0,Phoebe
1370,1017-1018,83,642,301,7.73494,0.468847,29,29,0,0,0.0124611,0,0,0,0,Chandler
1371,1017-1018,112,719,276,6.41964,0.383866,36,37,0,0,0.0111266,0,0,0,0,Ross
1372,1017-1018,84,687,281,8.17857,0.409025,32,41,0,0,0.0116448,0,0,0,0,Joey


In [286]:
# Most common words loop

master_array = [] #append results to array to create data frame
mostcommon = {}
mostcommon["Monica"] = {}
mostcommon["Rachel"] = {}
mostcommon["Phoebe"] = {}
mostcommon["Chandler"] = {}
mostcommon["Ross"] = {}
mostcommon["Joey"] = {}
for filename in os.listdir('season/'):
    if filename.endswith(".html"):
        split_name = filename.split('.') #obtain the season and episodes
        season = split_name[0]   
        episode = split_name[1]

        master_array.append([season, episode])
    else:
        pass
for num, ep in enumerate(master_array):
    filename=master_array[num][0]+'.'+master_array[num][1]

    f = open('season/'+filename, 'r')
    data = f.read()

    soup = BeautifulSoup(data)
    readsoup = BeautifulSoup(data)

    soup = soup.get_text().translate(str.maketrans({'\n': ' ', '\xa0': ''}))
    pattern = re.compile(r'\s(?=\w+(?=:))') # store the regex
    result = re.split(pattern, soup) # split the script where our pattern matched (pink dot)


    episode_array=[]
    for item in result:
        split_line = item.split(': ')
        try:
            character = split_line[0]
            speech = split_line[1]
            episode_array.append([character, speech])
        except:
            pass
    
    df = pd.DataFrame(episode_array, columns = ['Person','Said'])

    df['Person'].replace({'CHAN':'Chandler','CHANDLER':'Chandler', 'Chandlers':'Chandler',
                        'JOEY':'Joey',
                        'MNCA':'Monica','MONICA':'Monica',
                        'PHOE':'Phoebe','PHOEBE':'Phoebe', 'Pheebs':'Phoebe',
                        'Rache':'Rachel','RACHEL':'Rachel', 'RACH':'Rachel',
                        'ROSS':'Ross'},inplace=True)

    curved = re.compile("(\(.*?\))")
    brackets = re.compile("(\[.*?\])")
    scene = re.compile("\[Scene")
    for j,i in enumerate(df['Said']):
        df.iloc[j][1] =re.sub(curved, '', i)

    for j,i in enumerate(df['Said']):
        df.iloc[j][1] =re.sub(brackets, '', i)
        
    for j,i in enumerate(df['Said']):
        df.iloc[j][1] =re.sub(scene, '', i)

    worddicts = {}
    worddicts['Monica'] = {}
    worddicts["Rachel"]={}
    worddicts["Phoebe"]={}
    worddicts["Chandler"]={} 
    worddicts["Ross"]={} 
    worddicts["Joey"]={}
    for i, j in enumerate(df["Person"]):
        if j in worddicts:
            wordlist = Counter((x.rstrip(punctuation).lower() for x in df.iloc[i][1].split()))
            for word in wordlist:
                try:
                    worddicts[j][word] = worddicts[j][word]+1
                except KeyError:
                    worddicts[j][word] = 1
                    
    for p in namedict:
        for key in worddicts[p].keys():
            try:
                mostcommon[p][key] = mostcommon[p][key]+worddicts[p][key]
            except:
                mostcommon[p][key] = worddicts[p][key]

In [214]:
# Commonloop ANVÄND EJ
    commonwords = {}
    commonwords['Monica']={}
    commonwords["Rachel"]={}
    commonwords["Phoebe"]={}
    commonwords["Chandler"]={} 
    commonwords["Ross"]={} 
    commonwords["Joey"]={}
    for b in namedict:
        for c in namedict:
            ucommonwords[b][c] = 0


    for p in namedict:
        for person in namedict:
            for word in nlargest(30, worddicts[person], key=worddicts[person].get):
                if word in nlargest(30, mostcommon[p], key=mostcommon[p].get):
                    try:
                        commonwords[p][person] = commonwords[p][person]+worddicts[person][word]
                    except:
                        commonwords[p][person] = worddicts[p][word]
                else:
                    pass

In [150]:
# ANVÄND EJ
import itertools as it
uniqueWord = {}
uniqueWord["Monica"] = {}
uniqueWord["Rachel"] = {}
uniqueWord["Phoebe"] = {}
uniqueWord["Chandler"] = {}
uniqueWord["Ross"] = {}
uniqueWord["Joey"] = {}
numberOfWords = 2

for person in worddicts:
    per = [person]
    
    # Creates a list with all peoples words except from "person"
    tempExclusive = list(it.chain.from_iterable([list(mostcommon[i].keys()) for i in list(worddicts.keys()) if i not in per]))
    uniqueWord[person] = dict([(i, mostcommon[person][i]) for i in mostcommon[person] if i not in tempExclusive])

    

In [287]:
import itertools as it
uniqueWord = {}
uniqueWord["Monica"] = {}
uniqueWord["Rachel"] = {}
uniqueWord["Phoebe"] = {}
uniqueWord["Chandler"] = {}
uniqueWord["Ross"] = {}
uniqueWord["Joey"] = {}
numberOfWords = 30 # Antalet ord i listan "n"
mult = 2 # 


for person in worddicts:
    
    more = 0
    
    while len(uniqueWord[person]) < numberOfWords:
        
        per = [person]
        # Skapar en lista med personens more + n-mest populära ord
        personTop = nlargest(numberOfWords+more, mostcommon[person], key=mostcommon[person].get)
        
        # Skapar en lista med övriga karaktärers n*mult + more populära ord
        tempExclusive = set(it.chain.from_iterable([nlargest((mult*numberOfWords)+more, mostcommon[i], key=mostcommon[i].get) for i in list(worddicts.keys()) if i not in per]))
        tempExclusive = list(tempExclusive)
        
        # Plockar ut de ord som endast återfinnes i personTop, ej tempExclusive
        uniqueWord[person] = dict([(i, mostcommon[person][i]) for i in personTop if i not in tempExclusive])
        
        # Utöka listan med ett element
        more +=1

In [293]:
import sklearn.linear_model as skl_lm
import sklearn.model_selection as skl_ms
import numpy as np
import pandas as pd
import sklearn.discriminant_analysis as skl_da
import sklearn.preprocessing as skl_pre
import math
#epinfo = pd.read_csv('epinfo.csv')
epinfo = epinfo.transpose()
features = [#'Episode', 
            'Number of lines', 
            'Number of words', 
            'Different words used',
            'Words per line', 
            'Different words ratio',
            'Exclamation marks', 
            'Question marks', 
            'Word',
            "Most common Monica",
            "Most common Rachel",
            "Most common Phoebe",
            "Most common Chandler",
            "Most common Ross",
            "Most common Joey",
            #'Most common 10',
            #'Most common 30', 
            #'Most common 50', 
             ]
#encode = ['Most common 10',
#            'Most common 30', 
#            'Most common 50', 
#            ]
X = epinfo[list(features)]
#X2 = epinfo[list(encode)]
#scaler = skl_pre.LabelEncoder.fit(X2)
#X = X1 + scaler.transform(X2)
Y = epinfo['Character']

X_train, X_test, Y_train, Y_test = skl_ms.train_test_split(X,Y, test_size = 0.1)

model = skl_lm.LogisticRegression(solver='newton-cg', multi_class='multinomial', max_iter=10000, 
                                  penalty='l2', fit_intercept = False)
#model = skl_da.QuadraticDiscriminantAnalysis()
model.fit(X_train, Y_train)
prediction = model.predict(X_test)
Y_hat = model.predict(X)
print(np.mean(prediction==Y_test))
print(pd.crosstab(prediction, Y_test))


0.26811594202898553
Character  Chandler  Joey  Monica  Phoebe  Rachel  Ross
row_0                                                  
Chandler          9     4       6       1       8    10
Joey              2     2       4       4       4     5
Monica            4     1       2       1       2     0
Phoebe            5     2       6      13       6     5
Rachel            2     2       3       2       7     1
Ross              5     2       1       2       1     4
